In [ ]:
ls /Users/mahtin/Dropbox/Icke/Work/somVar/testdata/EBdata/

In [1]:
# get the code
import sys
sys.path.append('../code')

# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
ebdata = f"{testdata}/EBdata"
pon_path = f"{testdata}/testpon"
static = f"{home}/Dropbox/Icke/Work/static"
shell_path = "../shell"
# squeezing all data and shell paths into config
# shell paths can be autogenerated from common shell path
mawk_tools = ["cleanpileup", "csv2bed", "pon2cols", "pile2count", "filterVar", "pon2tumor", ]
mawk_tool_dict = {tool:f"{shell_path}/{tool}.mawk" for tool in mawk_tools}


EBconfig = {
    "makeponlist": "../shell/makeponlist.sh",
    "pon_path": pon_path,
    "genome_split": f"{static}/genome/gatk/hg38/split",
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "count_dict": {0:"alt+", 1:"alt-", 2:"depth+", 3:"depth-"}
}
EBconfig.update(mawk_tool_dict)
EBconfig

{'makeponlist': '../shell/makeponlist.sh',
 'pon_path': '/Users/martinscience/Dropbox/Icke/Work/somVar/testdata/testpon',
 'genome_split': '/Users/martinscience/Dropbox/Icke/Work/static/genome/gatk/hg38/split',
 'MAPQ': 20,
 'Q': 25,
 'fit_pen': 0.5,
 'count_dict': {0: 'alt+', 1: 'alt-', 2: 'depth+', 3: 'depth-'},
 'cleanpileup': '../shell/cleanpileup.mawk',
 'csv2bed': '../shell/csv2bed.mawk',
 'pon2cols': '../shell/pon2cols.mawk',
 'pile2count': '../shell/pile2count.mawk',
 'filterVar': '../shell/filterVar.mawk',
 'pon2tumor': '../shell/pon2tumor.mawk'}

In [ ]:
from ebraw import bam2matrix
bam_file = f"{testdata}/bam/002_A.bam"
mut_file = f"{ebdata}/002.csv"
pon_list = "Pon_chr7short.txt"
chrom = "chr7"
df = bam2matrix(bam_file, mut_file, chrom, pon_list, EBconfig)
df[:3]

### EX|INport

In [1]:
# get the code
import sys
sys.path.append('../code')

# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
ebdata = f"{testdata}/EBdata"
pon_path = f"{testdata}/testpon"
static = f"{home}/Dropbox/Icke/Work/static"


# df.to_csv(f"{ebdata}/002.matrix", sep='\t', index=False)
df = pd.read_csv(f"{ebdata}/002.matrix", sep='\t')
df[:3]

,Chr,Start,End,Ref,Alt,Tumor:Alt=Depth,PON:ALT=Depth
0,chr7,680651,680651,T,G,0-1=28-2,0|0|0|0-5|2|3|3=129|92|148|166-10|6|16|8
1,chr7,35670232,35670232,C,T,4-3=13-4,102|0|0|0-18|0|0|0=102|92|167|229-18|24|42|44
2,chr7,72826970,72826970,G,A,2-5=10-10,40|0|0|99-26|0|0|50=126|189|187|291-80|100|110...


## matrix to AB

### get_count_df for matrix2EB

In [4]:
from ebutils import get_count_df
row = df.iloc[2,:]
row
cdf = get_count_df(row['PON:ALT=Depth'], EBconfig["count_dict"])
cdf

,alt+,alt-,depth+,depth-
0,40,26,126,80
1,0,0,189,100
2,0,0,187,110
3,99,50,291,124


In [5]:
from ebconvert import matrix2AB
from ebcore import fit_bb
bb_params = fit_bb(cdf, 0.5)
bb_params

'0.14402|1.0-0.16004|1.0'

In [6]:
matrix2AB(row, config=EBconfig)

'0.14402|1.0-0.16004|1.0'

### expand matrix2AB to df

In [7]:
df['ABparams'] = df.apply(matrix2AB, config=EBconfig, axis=1)
df[:3]

,Chr,Start,End,Ref,Alt,Tumor:Alt=Depth,PON:ALT=Depth,ABparams
0,chr7,680651,680651,T,G,0-1=28-2,0|0|0|0-5|2|3|3=129|92|148|166-10|6|16|8,0.1|2.30251-4.45091|8.45244
1,chr7,35670232,35670232,C,T,4-3=13-4,102|0|0|0-18|0|0|0=102|92|167|229-18|24|42|44,0.1|1.0-0.1|1.0
2,chr7,72826970,72826970,G,A,2-5=10-10,40|0|0|99-26|0|0|50=126|189|187|291-80|100|110...,0.14402|1.0-0.16004|1.0


## AB to EB

In [16]:
# make a row with the same values as in EBold for comparison
row_dict = {
    "Chr": ["chr7"],
    "Start": 680651,
    "Tumor:Alt=Depth": "0-5=12-42",
    "ABparams": "0.20509|23.07746-0.22661|2.47235"
}

test_row = pd.DataFrame().from_dict(row_dict).iloc[0]
test_row
from ebconvert import AB2EBscore
AB2EBscore(test_row, config=EBconfig)

0.234

In [13]:
df['EBscore'] = df.apply(AB2EBscore, config=EBconfig, axis=1)
df

,Chr,Start,End,Ref,Alt,Tumor:Alt=Depth,PON:ALT=Depth,ABparams,EBscore
0,chr7,680651,680651,T,G,0-1=28-2,0|0|0|0-5|2|3|3=129|92|148|166-10|6|16|8,0.1|2.30251-4.45091|8.45244,0.055
1,chr7,35670232,35670232,C,T,4-3=13-4,102|0|0|0-18|0|0|0=102|92|167|229-18|24|42|44,0.1|1.0-0.1|1.0,1.380
2,chr7,72826970,72826970,G,A,2-5=10-10,40|0|0|99-26|0|0|50=126|189|187|291-80|100|110...,0.14402|1.0-0.16004|1.0,0.871
3,chr7,75147946,75147946,C,T,1-2=9-9,21|46|31|14-12|13|10|2=84|130|120|57-57|84|86|42,25.6636|65.04676-4.79774|29.78674,0.154
4,chr7,99722302,99722302,A,G,4-1=15-3,0|0|0|0-0|0|0|0=144|182|211|217-19|31|50|31,0.1|2.34352-0.1|1.97866,1.422
5,chr7,100958075,100958075,C,T,1-0=161-69,29|23|38|32-8|4|7|5=1096|1476|1547|1792-554|77...,10.72193|498.81024-4.58621|508.20524,0.001
6,chr7,100959332,100959332,C,G,4-0=43-25,23|32|43|44-2|2|1|0=281|354|342|486-91|149|161...,29.12842|268.39286-1.25987|104.27625,0.042
7,chr7,100959344,100959344,G,C,5-0=46-20,30|36|50|60-3|2|1|0=290|369|340|504-108|148|16...,45.30331|339.90614-1.04185|73.94303,0.036
8,chr7,101033998,101033998,G,C,16-0=70-32,139|0|188|0-22|0|15|0=536|588|796|690-301|395|...,0.11793|1.0-0.11982|1.52596,0.338
9,chr7,101034004,101034004,G,A,17-0=70-33,142|0|187|0-24|0|16|0=553|579|803|691-299|414|...,0.11786|1.0-0.11808|1.43809,0.349
